In [126]:
# Import librairies
import numpy as np
import pandas as pd

# Define the function to simulate paths under the Heston-Hull-White model
def simulate_hhw_paths(
    S0: float,
    v0: float,
    r0: float,
    b0: float,
    kappa_v: float,
    theta_v: float,
    sigma_v: float,
    kappa_r: float,
    theta_r: float,
    sigma_r: float,
    rho_sv: float,
    rho_sr: float,
    rho_vr: float,
    T: float,
    N: int,
    M: int,
    seed: int = None
):
    """
    Simulate Monte Carlo paths under the Heston-Hull-White model and return a dict of DataFrames.

    Each key in the returned dict corresponds to a path index, and the DataFrame contains columns:
      - 'S': simulated asset prices
      - 'v': simulated variances
      - 'r': simulated short rates

    Returns:
    -------
    dfs : dict[int, pd.DataFrame]
        Dictionary mapping path index to a DataFrame of shape (M+1, 3).
    """
    dt = T / M

    # Build correlation matrix and its Cholesky factor
    corr = np.array([
        [1.0, rho_sv, rho_sr],
        [rho_sv, 1.0, rho_vr],
        [rho_sr, rho_vr, 1.0]
    ])
    L = np.linalg.cholesky(corr)

    # Arrays to store paths
    S = np.zeros((M + 1, N))
    v = np.zeros((M + 1, N))
    r = np.zeros((M + 1, N))
    S[0, :] = S0
    v[0, :] = v0
    r[0, :] = r0

    rng = np.random.default_rng(seed)
    
    for t in range(1, M + 1):
        # Generate correlated increments
        Z = rng.standard_normal((3, N))
        dW = (L @ Z) * np.sqrt(dt)
        dW1, dW2, dW3 = dW[0], dW[1], dW[2]

        # Full-truncation Euler for variance
        v_prev = np.maximum(v[t-1, :], 0.0)
        v[t, :] = (
            v[t-1, :]
            + kappa_v * (theta_v - v_prev) * dt
            + sigma_v * np.sqrt(v_prev) * dW2
        )
        v[t, :] = np.maximum(v[t, :], 0.0)

        # Hull-White short rate
        r[t, :] = (
            r[t-1, :]
            + kappa_r * (theta_r - r[t-1, :]) * dt
            + sigma_r * dW3
        )

        # Asset price log-Euler
        S[t, :] = (
            S[t-1, :]
            * np.exp((r[t-1, :] + b0 - 0.5 * v_prev) * dt + np.sqrt(v_prev) * dW1)
        )

    # Convert to DataFrames in a dict
    dfs = {}
    for i in range(N):
        dfs[i] = pd.DataFrame({
            'S': S[:, i],
            'v': v[:, i],
            'r': r[:, i]
        })
    return dfs


Questions: 
1. is SOFER + Const well implemented ?

In [127]:
# Example usage
params = {
    'S0': 100.0, 'v0': 0.04, 'r0': 0.03, 'b0': 0.02,
    'kappa_v': 2.0, 'theta_v': 0.04, 'sigma_v': 0.3,
    'kappa_r': 0.1, 'theta_r': 0.03, 'sigma_r': 0.01,
    'rho_sv': -0.7, 'rho_sr': 0.2, 'rho_vr': 0.1,
    'T': 1.0, 'N': 10000, 'M': 252, 'seed': 42
}

dfs = simulate_hhw_paths(**params)
print(f"Simulated {len(dfs)} paths over {params['M']} steps.")

# Display first few rows of path 0
print(dfs[0].head())

Simulated 10000 paths over 252 steps.
            S         v         r
0  100.000000  0.040000  0.030000
1  100.396597  0.039670  0.029114
2   99.142127  0.039919  0.029243
3  100.135207  0.032969  0.029340
4   98.380058  0.032907  0.029348


In [ ]:
df_temp = dfs[0].copy()
# Volatility target settings
lambda_s = 0.94
lambda_l = 0.97
target_volatility = 0.1
max_k = 1.5
lag = 2
initial_risk_control_index_value = 100.0
initial_realized_variance = 0.04/252
interest_rate_basis = 0.02963
df_temp['risky_asset_returns'] = df_temp['S'].pct_change()
variance_s = list()
variance_l = list()
for i in range(df_temp.shape[0]):
    if i == 0:
        variance_s.append(initial_realized_variance)
        variance_l.append(initial_realized_variance)
    else:
        r = df_temp['risky_asset_returns'].loc[i]**2
        variance_s.append(
            (1 - lambda_s) * r + lambda_s * variance_s[i - 1]
        )
        variance_l.append(
            (1 - lambda_l) * r + lambda_l * variance_l[i - 1]
        )
df_temp['variance_s'] = variance_s
df_temp['variance_l'] = variance_l
df_temp['volatility_l_annualized'] = np.sqrt(df_temp['variance_l'] * 252)
df_temp['volatility_s_annualized'] = np.sqrt(df_temp['variance_s'] * 252)
df_temp['realized_volatility'] = df_temp[['volatility_l_annualized', 'volatility_s_annualized']].max(axis=1)
df_temp['realized_volatility'] = df_temp['realized_volatility'].shift(lag)
df_temp['realized_volatility'] = df_temp['realized_volatility'].bfill()
df_temp['K'] = (target_volatility / df_temp['realized_volatility']).clip(upper=max_k)
df_temp['interest_rate'] = (df_temp['r'] + interest_rate_basis).shift(1)
df_temp['risk_control_index_returns'] = df_temp['K'] * (df_temp['risky_asset_returns'] - df_temp['interest_rate']/252)
df_temp['risk_control_index_value'] = df_temp['risk_control_index_returns'].fillna((0)).add(1).cumprod().multiply(initial_risk_control_index_value)

In [139]:
df_temp

,S,v,r,risky_asset_returns,variance_s,variance_l,volatility_l_annualized,volatility_s_annualized,realized_volatility,K,interest_rate,risk_control_index_returns,risk_control_index_value
0,100.000000,0.040000,0.030000,NaN,0.000159,0.000159,0.200000,0.200000,0.200000,0.500000,NaN,NaN,100.000000
1,100.396597,0.039670,0.029114,0.003966,0.000150,0.000154,0.197279,0.194519,0.200000,0.500000,0.059630,0.001865,100.186467
2,99.142127,0.039919,0.029243,-0.012495,0.000151,0.000154,0.197311,0.194752,0.200000,0.500000,0.058744,-0.006364,99.548867
3,100.135207,0.032969,0.029340,0.010017,0.000147,0.000153,0.196271,0.192794,0.197279,0.506897,0.058873,0.004959,100.042533
4,98.380058,0.032907,0.029348,-0.017528,0.000157,0.000157,0.199221,0.198959,0.197311,0.506814,0.058970,-0.009002,99.141958
...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,108.005649,0.066970,0.021705,-0.022264,0.000236,0.000235,0.243291,0.243853,0.242690,0.412049,0.050917,-0.009257,99.113307
249,106.532666,0.069385,0.022772,-0.013638,0.000233,0.000233,0.242530,0.242298,0.239076,0.418276,0.051335,-0.005790,98.539474
250,109.260066,0.068552,0.022598,0.025602,0.000258,0.000246,0.249021,0.255140,0.243853,0.410084,0.052402,0.010413,99.565615
251,111.383230,0.059244,0.022469,0.019432,0.000265,0.000250,0.251009,0.258650,0.242530,0.412320,0.052228,0.007927,100.354855


In [155]:
compute_risk_control_index(dfs)

,0_risk_control_index,1_risk_control_index,2_risk_control_index,3_risk_control_index,4_risk_control_index,5_risk_control_index,6_risk_control_index,7_risk_control_index,8_risk_control_index,9_risk_control_index,...,9990_risk_control_index,9991_risk_control_index,9992_risk_control_index,9993_risk_control_index,9994_risk_control_index,9995_risk_control_index,9996_risk_control_index,9997_risk_control_index,9998_risk_control_index,9999_risk_control_index
0,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,...,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
1,100.186467,99.343189,100.469160,100.590217,98.779923,99.180421,100.074728,99.795280,99.983538,99.459568,...,100.262611,98.849948,99.662243,100.434776,99.600598,101.025302,99.954333,100.047858,100.707772,99.879056
2,99.548867,99.305322,99.683790,100.152010,98.608241,98.899704,100.549387,99.348081,98.533980,98.353561,...,100.189144,100.027328,100.624538,100.993393,99.003298,100.372026,99.280641,101.031689,100.138771,99.620001
3,100.042533,99.101262,100.630527,99.520701,97.625963,97.713100,101.071547,98.912004,98.438918,99.119658,...,100.829028,100.030206,100.532260,101.540729,98.813418,99.922756,100.017226,100.078139,100.614418,98.755957
4,99.141958,99.858712,99.439467,100.543817,97.464858,97.954371,101.750281,98.513382,98.772218,98.465823,...,100.507228,99.004529,99.554915,101.252507,98.266694,100.597610,100.493330,101.071990,100.502373,99.249163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
248,99.113307,93.780670,85.158875,111.204366,94.505363,85.946203,118.341791,97.170623,112.320527,90.912791,...,92.409062,81.880666,98.322236,94.543082,103.556819,92.521606,110.262100,105.309844,95.843445,109.937463
249,98.539474,93.660276,85.071802,110.541454,94.263588,85.837957,119.717873,97.175871,112.214620,89.812889,...,92.921571,81.299015,98.280894,94.439173,103.258866,92.837876,109.842252,106.199722,95.399782,110.301594
250,99.565615,93.645997,85.400064,110.154057,95.317110,86.678178,119.712406,96.647374,111.144130,88.884291,...,93.573948,81.772741,98.097512,95.252447,102.417755,92.831259,109.032414,106.204789,95.014879,110.977638
251,100.354855,93.771481,84.660716,109.698499,94.596542,86.649792,119.522419,96.015247,111.448615,89.125622,...,93.603790,82.933409,97.901215,94.749639,101.676145,92.830987,109.073892,107.112990,95.508939,110.174924


In [143]:
import numpy as np
import pandas as pd

def compute_risk_control_index(
    dfs: dict,
    lambda_s: float = 0.94,
    lambda_l: float = 0.97,
    target_volatility: float = 0.1,
    max_k: float = 1.5,
    lag: int = 2,
    initial_risk_control_index_value: float = 100.0,
    initial_realized_variance: float = 0.04 / 252,
    interest_rate_basis: float = 0.02963
) -> pd.DataFrame:
    """
    Compute the risk control index for a collection of DataFrames.

    Parameters
    ----------
    dfs : dict of pd.DataFrame
        A dict mapping identifiers to DataFrames. Each DataFrame must contain columns:
        - 'S' : prices of the risky asset
        - 'r' : (continuous) returns or yields (used to derive interest_rate)
    lambda_s : float
        Decay factor for the short-run variance estimate.
    lambda_l : float
        Decay factor for the long-run variance estimate.
    target_volatility : float
        Annualized target volatility (e.g., 0.1 for 10%).
    max_k : float
        Maximum leverage factor K.
    lag : int
        Number of periods to lag the realized_volatility before computing K.
    initial_risk_control_index_value : float
        Starting level of the risk control index.
    initial_realized_variance : float
        Starting variance (daily) for the EWMA calculations.
    interest_rate_basis : float
        Constant added to the asset's return to approximate the risk-free rate.

    Returns
    -------
    pd.DataFrame
        A DataFrame whose columns are the risk control index time series for each
        input DataFrame (column names = dict keys), indexed by the original dates.
    """
    result_series = {}

    for name, df in dfs.items():
        df_temp = df.copy().sort_index()

        # 1. Compute risky asset returns
        df_temp['risky_asset_returns'] = df_temp['S'].pct_change()

        # 2. EWMA variances
        variance_s = []
        variance_l = []
        for i, r2 in enumerate((df_temp['risky_asset_returns']**2).fillna(0)):
            if i == 0:
                variance_s.append(initial_realized_variance)
                variance_l.append(initial_realized_variance)
            else:
                # short-run
                vs = (1 - lambda_s) * r2 + lambda_s * variance_s[i - 1]
                variance_s.append(vs)
                # long-run (note: using variance_s lagged per spec)
                vl = (1 - lambda_l) * r2 + lambda_l * variance_l[i - 1]
                variance_l.append(vl)

        # align lengths
        #variance_l.insert(0, initial_realized_variance)
        df_temp['variance_s'] = variance_s
        df_temp['variance_l'] = variance_l

        # 3. Annualized vols
        df_temp['volatility_s_annualized'] = np.sqrt(df_temp['variance_s'] * 252)
        df_temp['volatility_l_annualized'] = np.sqrt(df_temp['variance_l'] * 252)

        # 4. Realized volatility (max of the two), lagged and back-filled
        df_temp['realized_volatility'] = df_temp[['volatility_s_annualized',
                                                  'volatility_l_annualized']].max(axis=1)
        df_temp['realized_volatility'] = df_temp['realized_volatility'].shift(lag).bfill()

        # 5. Leverage factor K, capped at max_k
        df_temp['K'] = (target_volatility / df_temp['realized_volatility']).clip(upper=max_k)

        # 6. Interest rate (shifted by 1)
        df_temp['interest_rate'] = (df_temp['r'] + interest_rate_basis).shift(1)

        # 7. Risk control index returns
        df_temp['risk_control_index_returns'] = (
            df_temp['K'] * (df_temp['risky_asset_returns'] - df_temp['interest_rate'] / 252)
        )

        # 8. Cumulative index level
        df_temp['risk_control_index_value'] = (
            (1 + df_temp['risk_control_index_returns'].fillna(0))
            .cumprod()
            * initial_risk_control_index_value
        )

        # save the series
        result_series[name] = df_temp['risk_control_index_value']

    # combine into one DataFrame
    result_df = pd.concat(result_series, axis=1)
    result_df.columns = [f"{name}_risk_control_index" for name in result_series]

    return result_df
